# Plots for testing resetting the flow

Michael J. Williams 2023

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from nessai.plot import plot_indices

from thesis_utils import conf
from thesis_utils.plotting import set_plotting, save_figure
from thesis_utils.io import load_hdf5

set_plotting()

In [ ]:
no_reset_path = "outdir/no_reset_v3/result.hdf5"
reset_path = "outdir/reset_4_v3/result.hdf5"

In [ ]:
no_reset_result = load_hdf5(no_reset_path)

In [ ]:
reset_result = load_hdf5(reset_path)

In [ ]:
history = reset_result["history"]
history_no_reset = no_reset_result["history"]

Check the insertion indices plots.

In [ ]:
fig = plot_indices(reset_result["insertion_indices"], nlive=2000)
plt.show()

In [ ]:
fig = plot_indices(no_reset_result["insertion_indices"], nlive=2000)
plt.show()

True log-evidence for 50-dimensional Gaussian

In [ ]:
ln_Z = -50 * np.log(20)
print(f"Analytic log-evidence: {ln_Z}")

Print out summary of results

In [ ]:
print("No resetting")
print(f"Log-evidence = {no_reset_result['log_evidence']} +/- {no_reset_result['log_evidence_error']}")
print(f"Likelihood evaluations = {no_reset_result['total_likelihood_evaluations']}")
print(f"Likelihood time [s] = {no_reset_result['likelihood_evaluation_time']}")
print(f"Population time [min] = {no_reset_result['population_time'] / 60 }")
print("\n")
print("Resetting")
print(f"Log-evidence = {reset_result['log_evidence']} +/- {reset_result['log_evidence_error']}")
print(f"Likelihood evaluations = {reset_result['total_likelihood_evaluations']}")
print(f"Likelihood time [s] = {reset_result['likelihood_evaluation_time']}")
print(f"Population time [min] = {reset_result['population_time'] / 60 }")

In [ ]:
fig, axs = plt.subplots(4, 1, sharex=True)

its = history["iterations"]
dtrain = history["training_iterations"][1:] - history["training_iterations"][:-1]

its_no_reset = history_no_reset["iterations"]
dtrain_no_reset = history_no_reset["training_iterations"][1:] - history_no_reset["training_iterations"][:-1]

axs[0].plot(its, history["mean_acceptance"])
axs[0].plot(its_no_reset, history_no_reset["mean_acceptance"], ls=conf.linestyles[1])
axs[0].set_ylim([0, 1])
axs[0].set_ylabel("Acceptance")

axs[1].plot(
    history["training_iterations"],
    np.arange(len(history["training_iterations"])),
)
axs[1].plot(
    history_no_reset["training_iterations"],
    np.arange(len(history_no_reset["training_iterations"])),
    ls=conf.linestyles[1]
)
axs[1].set_ylim(0, len(history["training_iterations"]))

axs[1].set_ylabel("Cumulative \ntraining count")

axs[2].plot(history["training_iterations"][1:], dtrain)
axs[2].plot(
    history_no_reset["training_iterations"][1:], dtrain_no_reset,
    ls=conf.linestyles[1]
)
axs[2].set_ylabel("Iterations \nbetween training")
axs[2].set_ylim([0, 1500])

axs[3].plot(
    history["population"]["iterations"],
    history["population"]["acceptance"],
    ls=conf.linestyles[0],
)
axs[3].plot(
    history_no_reset["population"]["iterations"],
    history_no_reset["population"]["acceptance"],
    ls=conf.linestyles[1],
)
axs[3].set_ylabel("Rejection sampling \nacceptance")

axs[3].set_yscale("log")
axs[3].set_ylim([5e-4, 1])

for ax in axs:
    ax.fill_betweenx(
        y=ax.get_ylim(),
        x1=0,
        x2=history["training_iterations"][0],
        alpha=0.25,
        zorder=-1,
        color="gray",
        lw=0.0
    )
    ax.set_xlim([0, its[-1]])

axs[-1].set_xlabel("Iteration")

plt.tight_layout()
save_figure(fig, "nessai_reset_diagnostics", "figures")
plt.show()